# Cuda语义
`作者：Tina`
`时间：2018-05-07`

[torch-cuda](https://pytorch.org/docs/stable/cuda.html#module-torch.cuda)用于设置和运行CUDA操作。它会跟踪当前选中的GPU，而你分配的所有CUDA张量将默认在该设备上创建。选择的设备可以用[torch.cuda.device](https://pytorch.org/docs/stable/cuda.html#torch.cuda.device)上下文管理器来改变。

然而，一旦一个张量被分配，你就可以对它进行操作，不管选择的设备是什么，结果总是会被放置在与张量相关的那个设备上。

默认情况下，交叉gpu操作是不允许的，除了[copy_()](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.copy_)以及其他具有复制功能的方法，例如[to()](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.to)和[cuda()](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.cuda)。除非你启用点对点内存访问，否则任何在不同设备上操作张量的尝试都会产生错误。

下面是一个例子：

In [10]:
import torch
cuda = torch.device('cuda')     # 默认的CUDA设备
cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')  # GPU 1 (these are 0-indexed)

x = torch.tensor([1., 2.], device=cuda0)
# x.device is device(type='cuda', index=0)
y = torch.tensor([1., 2.]).cuda()
# y.device is device(type='cuda', index=0)

with torch.cuda.device(1):
    # 在GPU1上分配一个张量
    a = torch.tensor([1., 2.], device=cuda)

    # 将一个张量从CPU转移到GPU 1
    b = torch.tensor([1., 2.]).cuda()
    # a.device和b.device都是(type='cuda', index=1)

    # 也可以使用``Tensor.to`` 去转移一个tensor:
    b2 = torch.tensor([1., 2.]).to(device=cuda)
    # b.device and b2.device are device(type='cuda', index=1)

    c = a + b
    # c.device is device(type='cuda', index=1)
    print("c.device is: ", c.device)
    z = x + y
    # z.device is device(type='cuda', index=0)
    print("z.device is: ", z.device)

    # 即使在一个上下文中，也可以指定设备
    # (或者调用.cuda，参数转入一个索引)
    d = torch.randn(2, device=cuda0)
    e = torch.randn(2).to(cuda0)
    f = torch.randn(2).cuda(cuda0)
    print("d.device is :", d.device)
    # d.device, e.device, and f.device are all device(type='cuda', index=0)

c.device is:  cuda:1
z.device is:  cuda:0
d.device is : cuda:0


## 异步执行(Asynchronous execution)
默认情况下，GPU操作是异步的。当你调用一个使用GPU的函数时，操作会被排队到特定的设备上，但会在稍后执行。这允许我们并行执行更多的计算，包括在CPU或其他GPU上的操作。

一般来说，异步计算的效果对调用者来说是不可见的，因为：
- 每个设备按照操作的排队顺序去执行
- 当在CPU和GPU之间复制数据或者在两个GPU之间复制数据时，PyTorch自动执行必要的同步。因此，计算将继续进行，就好像每个操作都是同步执行的。

你可以通过设置环境变量`CUDA_LAUNCH_BLOCKING=1`来强制同步计算。当GPU上出现错误时，这是很方便的。对于异步执行，这样的错误在实际执行操作之后才会被报告，所以堆栈跟踪没有显示操作被请求的位置。

作为一个例外，一些函数，例如[copy_()](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.copy_)允许一个显式的`asyns`参数，这样就可以让调用者在不必要的情况下绕过同步。另一个例外是CUDA流，下面解释。

### CUDA流
[CUDA流](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#streams)是一个属于特定设备的线性执行序列。通常不需要显式地创建一个CUDA流，默认情况下，每个设备都使用自己的“默认”流。每条流中的操作都按照它们创建的顺序进行序列化，但是来自不同流的操作可以在任何相对顺序中并发执行，除非使用显式同步函数，比如[synchronize()](https://pytorch.org/docs/stable/cuda.html#torch.cuda.synchronize)或者[wait_stream](https://pytorch.org/docs/stable/cuda.html#torch.cuda.Stream.wait_stream)

例如，以下代码是不正确的：

In [11]:
cuda = torch.device('cuda')
s = torch.cuda.stream()  # Create a new stream.
A = torch.empty((100, 100), device=cuda).normal_(0.0, 1.0)
with torch.cuda.stream(s):
    # sum() may start execution before normal_() finishes!
    B = torch.sum(A)

TypeError: stream() missing 1 required positional argument: 'stream'

“当前流”是默认流时，当数据移动时，PyTorch会自动执行必要的同步，如上所述。然而，在使用非默认流时，应确保正确的同步。

## 内存管理
PyTorch使用一个缓存内存分配器来加速内存分配，这允许在没有设备同步的情况下快速回收内存。然而，分配器管理的未使用内存仍将被显示，如同`nvidia-smi`那样。

你可以使用[memory_allocated()](https://pytorch.org/docs/stable/cuda.html#torch.cuda.memory_allocated)和[max_memory_allocated()](https://pytorch.org/docs/stable/cuda.html#torch.cuda.max_memory_allocated)监控张量所占用的内存。

使用[memory_cached()](https://pytorch.org/docs/stable/cuda.html#torch.cuda.memory_cached)和[max_memory_cached()](https://pytorch.org/docs/stable/cuda.html#torch.cuda.max_memory_cached)来监控由缓存分配器管理的内存。

调用[empty_cache()](https://pytorch.org/docs/stable/cuda.html#torch.cuda.empty_cache)可以释放PyTorch中所有未使用的缓存内存，以便其他GPU应用程序使用这些内存。然而，张量占用的GPU内存将不会被释放，因此它不能增加用于PyTorch的GPU内存量。


## 最佳实践
### 设备无关代码(Device-agnostic code)
由于PyTorch的结构，你可能需要显式地编写与设备（CPU或GPU）无关的代码。一个例子可能是创建一个新的张量作为一个循环神经网络的初始隐藏状态。

第一步是确定是否应该使用GPU。一个常见的模式是使用Python的`argparse`模块读取用户参数，并用一个可以禁用CUDA的标志，与[is_available()](https://pytorch.org/docs/stable/cuda.html#torch.cuda.is_available)结合使用。下面例子中，`args.device`结果是一个可以用来将张量移动到CPU或CUDA的`torch.device`对象。

In [17]:
import argparse
import torch

parser = argparse.ArgumentParser(description='PyTorch Example')
parser.add_argument('--disable-cuda', action='store_true', help='Disable CUDA')
# args = parser.parse_args() # 官方代码，这里有问题，修改如下
args = parser.parse_args(['--disable-cuda'])
args.device = None
if not args.disable_cuda and torch.cuda.is_available():
    args.device = torch.device('cuda')
else:
    args.device = torch.device('cpu')

现在有了`args.device`，我们可以用它来在想要的设备上创建一个张量。代码如下，这里

In [19]:
x = torch.empty((8, 42), device=args.device)
print(x.device)
# net = Network().to(device=args.device)

cpu


这可以在许多情况下使用，以产生设备无关的代码。下面是使用`dataloader`的一个例子：

```python
cuda0 = torch.device('cuda:0')  # CUDA GPU 0
for i, x in enumerate(train_loader):
    x = x.to(cuda0)
```

当在一个系统上使用多个GPU时，可以使用`CUDA_VISIBLE_DEVICES`环境标志来管理PyTorch可使用的GPU。如上所述，为了手动控制一个张量创建在哪一个GPU上，最好的做法是使用一个[torch.cuda.device](https://pytorch.org/docs/stable/cuda.html#torch.cuda.device)上下文管理器。

In [21]:
print("Outside device is 0")  # On device 0 (default in most scenarios)
with torch.cuda.device(1):
    print("Inside device is 1")  # On device 1
print("Outside device is still 0")  # On device 0

Outside device is 0
Inside device is 1
Outside device is still 0


如果你有一个张量，并且**想要在同一个设备上创造一个相同类型的新张量，那么你可以使用一个`torch.Tensor.new_*`方法**（见[torch.Tensor](https://pytorch.org/docs/stable/tensors.html#torch.Tensor)]）。然而前面提到的`torch.*`函数([creation-ops](https://pytorch.org/docs/stable/torch.html#tensor-creation-ops))取决于当前的GPU上下文和传入的属性参数。但`torch.Tensor.new_*`方法保存这个张量的设备和其他属性信息。

在前向传播过程中需要在内部创建新的张量时，在创建模块时，推荐的做法是：

In [23]:
cuda = torch.device('cuda')
x_cpu = torch.empty(2)
x_gpu = torch.empty(2, device=cuda)
x_cpu_long = torch.empty(2, dtype=torch.int64)

y_cpu = x_cpu.new_full([3, 2], fill_value=0.3)
print(y_cpu)

y_gpu = x_gpu.new_full([3, 2], fill_value=-5)
print(y_gpu)

y_cpu_long = x_cpu_long.new_tensor([[1, 2, 3]])
print(y_cpu_long)

tensor([[ 0.3000,  0.3000],
        [ 0.3000,  0.3000],
        [ 0.3000,  0.3000]])
tensor([[-5., -5.],
        [-5., -5.],
        [-5., -5.]], device='cuda:0')
tensor([[ 1,  2,  3]])


如果你想要创建一个与另一个张量有相同类型和尺寸的张量，并且用1或0填充它，可以使用[ones_like](https://pytorch.org/docs/stable/torch.html#torch.ones_like)和[zeros_like](https://pytorch.org/docs/stable/torch.html#torch.zeros_like)（也保存了Tensor的`torch.device`和`torch.dtype`）。

```python
x_cpu = torch.empty(2, 3)
x_gpu = torch.empty(2, 3)

y_cpu = torch.ones_like(x_cpu)
y_gpu = torch.zeros_like(x_gpu)
```

### 使用固定内存缓冲区
当副本来自固定 (页锁) 内存时, 主机到 GPU 的复制速度要快很多。CPU张量和存储开放了一个[pinmemory()](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.pin_memory)方法，它返回对象的副本，将数据放在一个固定的区域。

另外，一旦你固定了一个张量或存储，你可以使用异步的GPU拷贝，仅仅通过一个额外的`non_blocking=True`参数传给[cude()](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.cuda)，这可以用于重叠数据传输与计算。

你可以通过将`pinmemory=True`传递给它的构造器，从而使[DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)将batch返回到固定的内存中。

在0.1.9版本中，更大数量的GPU（8+）可能没有得到充分利用，然而, 这是一个已知的问题, 也正在积极开发中。

用[multiprocessing](https://pytorch.org/docs/stable/multiprocessing.html#module-torch.multiprocessing)来使用CUDA模型有很多需要注意的地方，除非小心谨慎地满足数据处理要求，你的程序很可能会有不正确或未定义的行为。